In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ms-ouareth1/ultralytics/run_qrvwn3vd_model:v49', type='model')
artifact_dir = artifact.download()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Pc\_netrc
wandb: Currently logged in as: haithambadereddine-dz (haithambadereddine-dz-badereddine-haitham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact run_qrvwn3vd_model:v49, 197.82MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:2:38.8


In [7]:
from ultralytics import YOLO
import cv2
from pathlib import Path

# Set paths - using raw strings to handle Windows paths
MODEL_WEIGHTS = r"C:\Users\Pc\Desktop\model\artifacts\run_qrvwn3vd_model-v49\epoch49.pt"
VIDEO_PATH = r"C:\Users\Pc\Desktop\model\PPE_Part1.mp4"
OUTPUT_PATH = "detection_output.mp4"

def process_video_yolov8(model, video_path, output_path, conf_thresh=0.5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Set up output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Perform inference
        results = model(frame, verbose=False)
        
        # Draw boxes
        for result in results:
            boxes = result.boxes
            for box in boxes:
                if box.conf.item() > conf_thresh:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    label = f"{model.names[box.cls.item()]} {box.conf.item():.2f}"
                    cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Write and display
        out.write(frame)
        cv2.imshow('YOLOv8 Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load model
    model = YOLO(MODEL_WEIGHTS)
    
    # Process video
    print("Starting video processing...")
    process_video_yolov8(model, VIDEO_PATH, OUTPUT_PATH)
    print(f"Processing complete. Output saved to {OUTPUT_PATH}")

Starting video processing...
Processing complete. Output saved to detection_output.mp4


In [16]:
from ultralytics import YOLO
import cv2
import winsound  # For Windows sound alerts
import time
from pathlib import Path
import numpy as np

# Configuration
MODEL_WEIGHTS = Path(r"C:\Users\Pc\Desktop\model\artifacts\run_qrvwn3vd_model-v49\epoch49.pt")
VIDEO_PATH = Path(r"C:\Users\Pc\Desktop\model\PPE_Part1.mp4")
OUTPUT_PATH = "detection_output.mp4"
ALERT_CLASSES = ["Non-Helmet", "no-vest", "bare-arms"]  # Classes that should trigger alerts
ALERT_BEEP_FREQ = 1000  # Frequency in Hz
ALERT_BEEP_DUR = 200  # Duration in ms
ALERT_FLASH_COLOR = (0, 0, 255)  # Red flashing border
ALERT_FLASH_DURATION = 0.01  # Seconds

def alert_system(frame, alert_count):
    """Visual and audio alert system"""
    # Audio alert
    winsound.Beep(ALERT_BEEP_FREQ, ALERT_BEEP_DUR)
    
    # Visual alert (flashing border)
    flash_frame = frame.copy()
    border_width = 30
    cv2.rectangle(flash_frame, (0, 0), (flash_frame.shape[1], flash_frame.shape[0]), 
                 ALERT_FLASH_COLOR, border_width)
    
    # Show the alert frame briefly
    cv2.imshow('YOLOv8 Detection', flash_frame)
    cv2.waitKey(int(ALERT_FLASH_DURATION * 1000))
    
    # Display alert counter
    cv2.putText(frame, f"Alerts: {alert_count}", (20, 40), 
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

def process_video_with_alerts(model, video_path, output_path, conf_thresh=0.5):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    alert_count = 0
    last_alert_time = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Inference
        results = model(frame, verbose=False)
        
        # Process detections
        alert_triggered = False
        for result in results:
            boxes = result.boxes
            for box in boxes:
                if box.conf.item() > conf_thresh:
                    cls_name = model.names[box.cls.item()]
                    if cls_name.lower() in ALERT_CLASSES:
                        alert_triggered = True
                    
                    # Draw bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    color = (0, 255, 0)  # Green for normal detections
                    if cls_name.lower() in ALERT_CLASSES:
                        color = (0, 0, 255)  # Red for alert classes
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    label = f"{cls_name} {box.conf.item():.2f}"
                    cv2.putText(frame, label, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Trigger alert if needed (with cooldown)
        current_time = time.time()
        if alert_triggered and (current_time - last_alert_time) > ALERT_FLASH_DURATION:
            alert_count += 1
            alert_system(frame, alert_count)
            last_alert_time = current_time
        
        # Write and display
        out.write(frame)
        cv2.imshow('YOLOv8 Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    print("Loading YOLOv8 model...")
    model = YOLO(MODEL_WEIGHTS)
    
    print("Starting video processing with alert system...")
    process_video_with_alerts(model, VIDEO_PATH, OUTPUT_PATH)
    
    print(f"Processing complete. Output saved to {OUTPUT_PATH}")

Loading YOLOv8 model...
Starting video processing with alert system...


KeyboardInterrupt: 

In [20]:
from ultralytics import YOLO
import cv2
from pathlib import Path
import numpy as np

# Configuration
MODEL_WEIGHTS = Path(r"C:\Users\Pc\Desktop\model\artifacts\run_qrvwn3vd_model-v49\epoch49.pt")
VIDEO_PATH = Path(r"C:\Users\Pc\Desktop\model\PPE_Part1.mp4")
OUTPUT_PATH = "detection_output.mp4"
ALERT_CLASSES = ["Non-Helmet", "no-vest", "bare-arms"]  # Classes that should trigger alerts
CONFIDENCE_THRESHOLD = 0.5

def process_video_with_counter(model, video_path, output_path):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    # Initialize counters
    class_counts = {cls: 0 for cls in ALERT_CLASSES}
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Reset counts for current frame
        current_frame_counts = {cls: 0 for cls in ALERT_CLASSES}
        
        # Inference
        results = model(frame, verbose=False)
        
        # Process detections
        for result in results:
            boxes = result.boxes
            for box in boxes:
                if box.conf.item() > CONFIDENCE_THRESHOLD:
                    cls_name = model.names[box.cls.item()].lower()
                    
                    # Update counts if class is in our alert classes
                    if cls_name in ALERT_CLASSES:
                        class_counts[cls_name] += 1
                        current_frame_counts[cls_name] += 1
                    
                    # Draw bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    color = (0, 255, 0)  # Green for normal detections
                    if cls_name in ALERT_CLASSES:
                        color = (0, 0, 255)  # Red for alert classes
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    label = f"{cls_name} {box.conf.item():.2f}"
                    cv2.putText(frame, label, (x1, y1-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Display current frame counts
        y_offset = 40
        for cls, count in current_frame_counts.items():
            if count > 0:
                text = f"{cls}: {count} (Total: {class_counts[cls]})"
                cv2.putText(frame, text, (20, y_offset), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                y_offset += 30
        
        # Write and display
        out.write(frame)
        cv2.imshow('YOLOv8 Detection - PPE Counter', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    # Print final counts
    print("\nFinal Detection Counts:")
    for cls, count in class_counts.items():
        print(f"{cls}: {count}")

if __name__ == "__main__":
    print("Loading YOLOv8 model...")
    model = YOLO(MODEL_WEIGHTS)
    
    print("Starting video processing with PPE counter...")
    process_video_with_counter(model, VIDEO_PATH, OUTPUT_PATH)
    
    print(f"\nProcessing complete. Output saved to {OUTPUT_PATH}")

Loading YOLOv8 model...
Starting video processing with PPE counter...

Final Detection Counts:
Non-Helmet: 0
no-vest: 730
bare-arms: 5

Processing complete. Output saved to detection_output.mp4


In [1]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Using device: cuda
